In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#%reset
import numpy as np
import argparse
import re
import os, glob, datetime
#tensorflow==2.1.0
#tensorflow-estimator==2.1.0
import tensorflow as tf
import numpy as np
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract
from keras.models import Model, load_model
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import Adam
#import data_generator as dg
import keras.backend as K
import sys
sys.path.insert(0, "/modules")
sys.path.append('/utils')
print(sys.path)

In [ ]:
# Model configuration
patch_size=40
#img_width, img_height = patch_size, patch_size
img_width, img_height = 181, 217
bands=6
#batch_size = 150
batch_size = 25
print("hey there!")
no_epochs = 50
validation_split = 0.2
verbosity = 1
max_norm_value = 2.0
noise_factor = 0.55
number_of_visualizations = 6

In [ ]:
import numpy as np
from hsi_preprocess import *
from keras.models import load_model
# Load 3D data from CAVE datasets (numbered 1 to 25)
i=1
img3d=np.load('/data/HSI/CAVE/CAVE3d_'+format(i,"0>2d")+'.npy')
a=mri_3d_extract(img3d,patch_size=40,channels=1) # to extract 2d data (no. of channels=1)
#a=mri_3d_extract(img3d,patch_size=40,channels=6) # to extract 3d data (no. of channels=more)
for i in range(1,5):
  img3d=np.load('/data/HSI/CAVE/CAVE3d_'+format(i+1,"0>2d")+'.npy')
  patches=mri_3d_extract(img3d,patch_size=40,channels=1) # to extract 2d data (no. of channels=1)
  #patches=mri_3d_extract(img3d,patch_size=40,channels=6) # to extract 3d data (no. of channels=more)
  a=np.concatenate([a,patches])
print(a.shape)
##############################################################################################

#a=np.load('/content/drive/My Drive/VAE/data/HSI/CAVE/balloons_ms/balloons3DPatches.npy')

img_width,img_height,layers=patch_size,patch_size,bands
#Reshaping data  
if K.image_data_format() == 'channels_first':
    print("channel first format")
    # Considering 3D data
    #input_train = a.reshape(a.shape[0], 1, img_width, img_height,layers)
    #input_shape = (1, img_width, img_height,layers)
    
    # Considering 2D data
    input_train = a.reshape(a.shape[0], 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else:
    print("channel last format")
    
    # Considering 3D data
    #input_train = a.reshape(a.shape[0], img_width, img_height,layers,1)
    #input_shape = (img_width, img_height, layers, 1)
    
    #Considering 2D data
    input_train = a.reshape(a.shape[0], img_width, img_height,1)
    input_shape = (img_width, img_height, 1)

In [ ]:
#################### Normalize Input Data ##########################

print(a.shape)
print(input_train[0].shape)
numOfSamples= input_train.shape[0]
for i in range(numOfSamples):
    # Normalise data between [0,1] (for each sample)
    input_train[i] = (input_train[i] - np.min(input_train[i]))/np.ptp(input_train[i])
    # Normalise data between[0,255] (per sample) as integer: don't forget the parenthesis before astype(int)
    #input_train[i] = (255*(input_train[i] - np.min(input_train[i]))/np.ptp(input_train[i])).astype(int)        

In [ ]:
print(input_train[0].shape)

In [ ]:
# Add noise
from numpy.random import seed
from numpy.random import randint
from mri_preprocess import *

#print(randint(1,5,1))

pure_train = input_train

# for 2D data
noisy_input_train=add_gaussian_noise(input_train,10)
noisy_input_train=add_impulse_noise(noisy_input_train,20)

# for 3d data
#noisy_input_train=add_gaussian_noise3D(input_train,10)
#noisy_input_train=add_impulse_noise3D(noisy_input_train,20)
#noisy_input_test=noisy_input_test.astype('float32')
print(noisy_input_train.shape)

In [ ]:
#################### Normalize Noisy Data ##########################

print(a.shape)
print(input_train[0].shape)
numOfSamples= input_train.shape[0]
for i in range(numOfSamples):
    # Normalise data between [0,1] (for each sample)
    noisy_input_train[i] = (noisy_input_train[i] - np.min(noisy_input_train[i]))/np.ptp(noisy_input_train[i])
    # Normalise data between[0,255] (per sample) as integer: don't forget the parenthesis before astype(int)
    #input_train[i] = (255*(input_train[i] - np.min(input_train[i]))/np.ptp(input_train[i])).astype(int)  

#print(input_train[0].shape)      

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
#import tensorflow as tf
from keras.models import *
from keras.layers import  Input,Conv2D,Conv3D,BatchNormalization,Activation,Lambda,Subtract,concatenate,Add,merge

import keras.backend  as K
#from group_norm import GroupNormalization #
#from batch_renorm import BatchRenormalization
#def BRDNet(): #original format def BRDNet(), data is used to obtain the reshape of input data
#inpt = Input(shape=(None,None,None, 1)) #if the image is 3, it is color image. If the image is 1, it is gray color, 201807082123tcw
inpt = Input(shape=(None,None, 1)) #if the image is 3, it is color image. If the image is 1, it is gray color, 201807082123tcw
# 1st layer, Conv+relu
#x = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(inpt)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
#x = BatchRenormalization(axis=-1, epsilon=1e-3)(x)
x = BatchNormalization(axis=-1, epsilon=1e-3,renorm=1)(x)
x = Activation('relu')(x)
# 15 layers, Conv+BN+relu
for i in range(7):
    #x = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    #x = BatchRenormalization(axis=-1, epsilon=1e-3)(x)
    x = BatchNormalization(axis=-1, epsilon=1e-3,renorm=1)(x)
    x = Activation('relu')(x)   
# last layer, Conv 
for i in range(8):
    #x = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(x)
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    #x = BatchRenormalization(axis=-1, epsilon=1e-3)(x)
    x = BatchNormalization(axis=-1, epsilon=1e-3,renorm=1)(x)
    x = Activation('relu')(x) 
#x = Conv3D(filters=1, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(x) #gray is 1 color is 3
x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x) #gray is 1 color is 3
x = Subtract()([inpt, x])   # input - noise
#y = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(inpt)
y = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
#y = BatchRenormalization(axis=-1, epsilon=1e-3)(y)
y = BatchNormalization(axis=-1, epsilon=1e-3,renorm=1)(y)
y = Activation('relu')(y)
# 15 layers, Conv+BN+relu
for i in range(7):
    #y = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1),dilation_rate=(2,2,2), padding='same')(y)
    y = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1),dilation_rate=(2,2), padding='same')(y)
    y = Activation('relu')(y)   
#y = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(y)
y = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(y)
#y = BatchRenormalization(axis=-1, epsilon=1e-3)(y)
y = BatchNormalization(axis=-1, epsilon=1e-3,renorm=1)(y)
y = Activation('relu')(y) 
for i in range(6):
    #y = Conv3D(filters=64, kernel_size=(3,3,3), strides=(1,1,1),dilation_rate=(2,2,2), padding='same')(y)
    y = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1),dilation_rate=(2,2), padding='same')(y)
    y = Activation('relu')(y)
#y = Conv3D(filters=128, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(y)
y = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(y)
#y = BatchRenormalization(axis=-1, epsilon=1e-3)(y)
y = BatchNormalization(axis=-1, epsilon=1e-3,renorm=1)(y)
y = Activation('relu')(y)    
#y = Conv3D(filters=1, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(y)#gray is 1 color is 3
y = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(y)#gray is 1 color is 3
y = Subtract()([inpt, y])   # input - noise
o = concatenate([x,y],axis=-1)
#z = Conv3D(filters=1, kernel_size=(3,3,3), strides=(1,1,1), padding='same')(o)#gray is 1 color is 3
z = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(o)#gray is 1 color is 3
z=  Subtract()([inpt, z])
model = Model(inputs=inpt, outputs=z)
#model.summary()
#return model

In [ ]:
#from losses import ssim_loss
#from mymetrics import ssim_metric,psnr_metric,ssim_loss
from keras.losses import mean_squared_error, mean_absolute_error
from skimage.metrics import structural_similarity as ssim
import skimage.metrics
import pytorch_ssim
import torch
from torch.autograd import Variable
from skimage import measure
import math
def ssim_metric2d(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))

def ssim_metric3d(y_true, y_pred):
  return tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0,filter_size=5))

def psnr_metric(y_true, y_pred):
    """
    PSNR is Peek Signal to Noise Ratio, which is similar to mean squared error.
    It can be calculated as
    PSNR = 20 * log10(MAXp) - 10 * log10(MSE)
    When providing an unscaled input, MAXp = 255. Therefore 20 * log10(255)== 48.1308036087.
    However, since we are scaling our input, MAXp = 1. Therefore 20 * log10(1) = 0.
    Thus we remove that component completely and only compute the remaining MSE component.
    """
    return -10. * K.log(K.mean(K.square(y_pred - y_true)))
def psnr_met1(y_true, y_pred):
    max_pixel = 1.0
    return (10.0 * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true), axis=-1)))) / 2.303

# Loss functtion
def ssim_loss1(y_true, y_pred):
    myssim=tf.image.ssim(y_true, y_pred,1)
    s_3d_tf = tf.Session().run(myssim)
    return tf.reduce_mean((1.0-s_3d_tf)/2.0)

def ssim_loss2(y_true, y_pred):
    print(y_true.dtype)
    img1=torch.cuda.FloatTensor(y_true)
    img2=torch.cuda.FloatTensor(y_pred)
    #img1=tf.convert_to_tensor(y_true)
    #img2=tf.convert_to_tensor(y_pred)
    print(img1.dtype)
    print(img2.shape)
    ssim_3d_sk=pytorch_ssim.ssim3D(y_true, y_pred)
    #ssim_3d_sk = skimage.metrics.structural_similarity(y_true, y_pred, multichannel=False, gaussian_weights=True, data_range=1.0, use_sample_covariance=False)
    #ssim_3d_sk = ssim(img1, img2,win_size=3, gaussian_weights=True, data_range=1.0, use_sample_covariance=False)
    return tf.reduce_mean((1.0-ssim_3d_sk)/2.0)

# Loss functtion
def ssim_loss2d(y_true, y_pred):
    return tf.reduce_mean((1.0-tf.image.ssim(y_true, y_pred,1))/2.0)

def ssim_loss3d(y_true, y_pred):
    return tf.reduce_mean((1.0-tf.image.ssim_multiscale(y_true, y_pred,1,filter_size=7))/2.0)

def SSIMLoss(y_true, y_pred):
  return 1 - tf.reduce_mean(ssim(y_true, y_pred))

def psnr_met2(y_true, y_pred):
    max_pixel = 1.0
    return 10.0 * (1.0 / math.log(10)) * K.log((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))

def L2_SSIM_loss(y_true, y_pred):
  alpha=0.5
  return alpha*mean_squared_error(y_true, y_pred) + (1.0-alpha)* ssim_loss(y_true, y_pred) 

def L1_SSIM_loss(y_true, y_pred):
  alpha=0.5
  return alpha*mean_absolute_error(y_true, y_pred) + (1.0-alpha)* ssim_loss(y_true, y_pred) 

def total_content_loss2d(y_true,y_pred):
  alpha=0.3
  tv_weight=1e-5
  '''
  Total variation loss is used to keep the image locally coherent
   '''
  #assert K.ndim(x) == 4
  a = tf.square(y_pred[:, :-1, :-1, :] - y_pred[:, 1:, :-1, :])
  b = tf.square(y_pred[:, :-1, :-1, :] - y_pred[:, :-1, 1:, :])
  #a = tf.square(x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, 1:, : img_ncols - 1, :])
  #b = tf.square(x[:, : img_nrows - 1, : img_ncols - 1, :] - x[:, : img_nrows - 1, 1:, :])
  return tv_weight*K.sum(K.pow(a + b,1.25))+alpha*mean_squared_error(y_true,y_pred)+(1-alpha)*mean_absolute_error(y_true,y_pred)

def L2_L1_loss(y_true,y_pred):
  alpha=0.4
  return alpha*mean_squared_error(y_true,y_pred)+(1-alpha)*mean_absolute_error(y_true,y_pred)

def mse_TV(y_true,y_pred):
  tv_weight=1e-5
  mse_weight=0.5
  return mse_weight*mean_squared_error(y_true,y_pred)+tv_weight*total_variation_loss(y_pred)


In [ ]:
# Compile and fit data
from keras.callbacks import CSVLogger
from keras import losses
from keras import optimizers
from mymetrics import *
from ganModel import *
import keras
#from mymetrics import ssim_metric,psnr_met2,ssim_loss
csv_logger = CSVLogger('/logs/HSI-logs/logfile-RL2DG10I20_epoch200-L2L1TV-spl-renorm.log', separator=',', append=False)
checkpoint = keras.callbacks.ModelCheckpoint('/models/HSI/EnsRL2D-G10I20_{epoch:03d}_L2L1TV-spl-renorm.h5', period=20)
adam=optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=True)
#sgdm=optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=True)
#adam = optimizers.adam(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss=losses.mean_square_error, optimizer=adam)
#model=generator_model()
#residual_train.astype('float32')
#print(np.max(residual_train))
model.compile(optimizer=adam, loss=total_content_loss2d,metrics=[psnr_met2,ssim_metric2d])
model.fit(noisy_input_train, input_train,callbacks=[checkpoint,csv_logger],
               epochs=200,
             batch_size=30,
                validation_split=validation_split)
                

In [ ]:
model.save("/models/HSI/EnsRL2DG10I10.h5")